In [1]:
from pygraphblas import *
from pygraphblas.demo.gviz import draw, draw_op
import pygraphblas.descriptor
import csv
import sys



In [47]:
#Load data from CSV format
class DataLoader:
    
    def __init__(self, path):
        self.path = path
        
    def load_node(self, filename):
        filename = self.path + filename
        with open(filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
            original_ids = [row['id:ID'] for row in reader]
            id_mapping = {}
            for index in range(len(original_ids)):
                id_mapping[original_ids[index]] = index
            
        return original_ids, id_mapping

    def load_edge(self, filename, start_mapping, end_mapping, typ=BOOL, drop_dangling_edges=False):
        filename = self.path + filename
        with open(filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
            row_ids = []
            col_ids = []
            values = []
            for row in reader:
                start_id = row['id:START_ID']
                end_id = row['id:END_ID']
                if not drop_dangling_edges or (start_id in start_mapping and end_id in end_mapping):
                    row_ids.append(start_mapping[start_id])
                    col_ids.append(end_mapping[end_id])
                    values.append(1)
        
            edge_matrix = Matrix.from_lists(
            row_ids,
            col_ids,
            values,
            nrows=len(start_mapping), 
            ncols=len(end_mapping), 
            typ=typ)
            return edge_matrix

def print_data_dimensions(vertices, matrices):
    for vertex in vertices:
        print(f"dimension of {vertex} is {len(vertices[vertex])}")
    
    for matrix in matrices:
        print(f"dimension of {matrix} is {matrices[matrix].shape}")


    

In [104]:
'''
Check results here:
https://github.com/ftsrg/trainbenchmark/blob/master/trainbenchmark-tool/src/main/java/hu/bme/mit/trainbenchmark/benchmark/test/TrainBenchmarkTest.java#L139 
'''

path = 'trainbenchmark-repair-models-csv/'
loader = DataLoader(path)
data_size = 2

vertices = {}
mapping = {}
vertices['Route'], mapping['Route'] = loader.load_node(f'railway-repair-{data_size}-Route.csv')
vertices['SwitchPosition'], mapping['SwitchPosition'] = loader.load_node(f'railway-repair-{data_size}-SwitchPosition.csv')
vertices['Switch'], mapping['Switch'] = loader.load_node(f'railway-repair-{data_size}-Switch.csv')
vertices['Sensor'], mapping['Sensor'] = loader.load_node(f'railway-repair-{data_size}-Sensor.csv')
vertices['Segment'], mapping['Segment'] = loader.load_node(f'railway-repair-{data_size}-Segment.csv')
vertices['Semaphore'], mapping['Semaphore'] = loader.load_node(f'railway-repair-{data_size}-Semaphore.csv')

matrices = {}
matrices['follows'] = loader.load_edge(f'railway-repair-{data_size}-follows.csv', mapping['Route'], mapping['SwitchPosition'])
matrices['target'] = loader.load_edge(f'railway-repair-{data_size}-target.csv', mapping['SwitchPosition'], mapping['Switch'])
matrices['monitoredBySwitch'] = loader.load_edge(f'railway-repair-{data_size}-monitoredBy.csv', mapping['Switch'], mapping['Sensor'], drop_dangling_edges=True)
matrices['monitoredBySegment'] = loader.load_edge(f'railway-repair-{data_size}-monitoredBy.csv', mapping['Segment'], mapping['Sensor'], drop_dangling_edges=True)
matrices['monitoredBy'] = loader.load_edge(f'railway-repair-{data_size}-monitoredBy.csv', {**mapping['Segment'], **mapping['Switch']}, mapping['Sensor'], drop_dangling_edges=False)
matrices['requires'] = loader.load_edge(f'railway-repair-{data_size}-requires.csv', mapping['Route'], mapping['Sensor'])
matrices['connectsToSeg'] = loader.load_edge(f'railway-repair-{data_size}-connectsTo.csv', mapping['Segment'], mapping['Segment'], drop_dangling_edges=True)
matrices['connectsToTrackElem'] = loader.load_edge(f'railway-repair-{data_size}-connectsTo.csv', {**mapping['Segment'], **mapping['Switch']}, {**mapping['Segment'], **mapping['Switch']}, drop_dangling_edges=False)
matrices['exit'] = loader.load_edge(f'railway-repair-{data_size}-exit.csv', mapping['Route'], mapping['Semaphore'])
matrices['entry'] = loader.load_edge(f'railway-repair-{data_size}-entry.csv', mapping['Route'], mapping['Semaphore'])

print_data_dimensions(vertices, matrices)

#Uncomment to trace down specific Sensor
#selected_sensor_id = mapping['Sensor']['1692']

def route_sensor_violation_query(matrices):
    route_to_switch = matrices['follows'] @ matrices['target']
    route_to_sensor = route_to_switch @ matrices['monitoredBySwitch']
    return route_to_sensor.extract_matrix(matrices['requires'], desc=descriptor.ooco)


def connected_segments_query(matrices, vertices):
    monitoredBySegmentTransposed = matrices['monitoredBySegment'].transpose()
    res = monitoredBySegmentTransposed.dup()
    for _ in range(5):
        #I, J, V = res[[selected_sensor_id],:].to_lists()
        #print([vertices['Segment'][j] for j in J])
        res = res.mxm(matrices['connectsToSeg'], mask=monitoredBySegmentTransposed)
        
    I, J, V = res.to_lists()
    violating_sensor_ids = [vertices['Sensor'][i] for i in I]
    print('The IDs of the sensors that violate the constraints:')
    print(violating_sensor_ids)
    print(f'The total number of violations is {len(res)}')
              
def switch_monitored_query(matrices):
    return matrices['monitoredBy']

def semaphore_neighbor_query(matrices):    
    monitoredByTransposed = matrices['monitoredBy'].transpose()
    connectsToTrackElemTransposed = matrices['connectsToTrackElem'].transpose()
    requiresTransposed = matrices['requires'].transpose()

    route_to_trackelement1 = matrices['requires'] @ monitoredByTransposed
    route_to_trackelement2 = route_to_trackelement1 @ connectsToTrackElemTransposed
    route_to_sensor = route_to_trackelement2 @ matrices['monitoredBy']
    route_to_route_ = route_to_sensor @ requiresTransposed 
    route_to_route = route_to_route_.offdiag()
    route_to_semaphore = route_to_route @ matrices['exit']
    return route_to_semaphore.extract_matrix(mask=matrices['entry'], desc=descriptor.ooco)


semaphore_neighbor_query(matrices)

# connected_segments_query(matrices, vertices)
# route_sensor_violations_result = route_sensor_violation_query(matrices)
# print(len(route_sensor_violations_result))




#print(route_sensor_violations_result.to_string())
# result.to_string()

dimension of Route is 10
dimension of SwitchPosition is 67
dimension of Switch is 67
dimension of Sensor is 310
dimension of Segment is 1564
dimension of Semaphore is 10
dimension of follows is (10, 67)
dimension of target is (67, 67)
dimension of monitoredBySwitch is (67, 310)
dimension of monitoredBySegment is (1564, 310)
dimension of monitoredBy is (1631, 310)
dimension of requires is (10, 310)
dimension of connectsToSeg is (1564, 1564)
dimension of connectsToTrackElem is (1631, 1631)
dimension of exit is (10, 10)
dimension of entry is (10, 10)


<Matrix (10x10 : 27:BOOL)>